# Image features exercise

In this exercise we will show that we can improve our classification performance by training  classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [1]:
import random
import numpy as np
from camalab.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15., 12.) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [2]:
def get_CIFAR10_data(num_training=5000, num_validation=500, num_test=500):
  # Load the raw CIFAR-10 data
  cifar10_dir = 'camalab/datasets/cifar-10-batches-py' # you should change it to your own path, 
                                                      # or put the dataset to this path  
    
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir, 3)
  
  # Subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]

  return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print X_train.shape
print X_val.shape
print X_test.shape

(5000L, 32L, 32L, 3L)
(500L, 32L, 32L, 3L)
(500L, 32L, 32L, 3L)


## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for the bonus section.

You should import your `hog` and `color_histogram` functions. Both operate on a single
image and return a feature vector for that image. 

Your function should takes a set of images and a list of feature functions and evaluates each feature function on each image, storing the results in a matrix where each column is the concatenation of all feature vectors for a single image.

In [3]:
from scipy.ndimage import uniform_filter
import matplotlib.colors as colors

### extract_features

In [10]:
def extract_features(imgs, feature_fns, verbose=False):
  num_images = imgs.shape[0]
  if num_images == 0:
    return np.array([])

  # 确定第一个图像特征维度
  feature_dims = []
  first_image_features = []
  for feature_fn in feature_fns:
    feats = feature_fn(imgs[0].squeeze())
    assert len(feats.shape) == 1 #特征函数1维
    feature_dims.append(feats.size)
    first_image_features.append(feats)

  # 分配
  total_feature_dim = sum(feature_dims)
  imgs_features = np.zeros((num_images, total_feature_dim))
  imgs_features[0] = np.hstack(first_image_features).T

  # 提取其余图像特征
  for i in xrange(1, num_images):  #与range的差别，可以逐一替代
    idx = 0
    for feature_fn, feature_dim in zip(feature_fns, feature_dims):
      next_idx = idx + feature_dim
      imgs_features[i, idx:next_idx] = feature_fn(imgs[i].squeeze())
      idx = next_idx

  return imgs_features

### hog_feature

1）灰度化（将图像看做一个x,y,z（灰度）的三维图像）；

2）采用Gamma校正法对输入图像进行颜色空间的标准化（归一化）；目的是调节图像的对比度，降低图像局部的阴影和光照变化所造成的影响，同时可以抑制噪音的干扰；

3）计算图像每个像素的梯度（包括大小和方向）；主要是为了捕获轮廓信息，同时进一步弱化光照的干扰。

4）将图像划分成小cells（例如6*6像素/cell）；

5）统计每个cell的梯度直方图（不同梯度的个数），即可形成每个cell的descriptor；

6）将每几个cell组成一个block（例如3*3个cell/block），一个block内所有cell的特征descriptor串联起来便得到该block的HOG特征descriptor。

7）将图像image内的所有block的HOG特征descriptor串联起来就可以得到该image（你要检测的目标）的HOG特征descriptor了。这个就是最终的可供分类使用的特征向量了。

In [11]:
#假设每个block块都包含2*2个cell，默认步长也为2,设定cell的360度方向分成8个方向块
def hog_feature(im):

  im = np.dot(im[...,:3], [0.299, 0.587, 0.144])# 灰度图参数
  figure_x, figure_y = im.shape
  cell_x,cell_y = (8,8)
  grad_x = np.zeros((figure_x, figure_y))#初始化数组
  grad_y = np.zeros((figure_x, figure_y))
  grad = np.zeros((figure_x, figure_y, 2))

  for i in xrange(1, figure_x - 1):
    for j in xrange(1, figure_y - 1):
      grad_x[i, j] = im[i, j - 1] - im[i, j + 1]
      grad_y[i, j] = im[i + 1, j] - im[i - 1, j]
      grad_dir[i, j] = (np.arctan(grad_y[i, j] / (grad_x[i, j] + 1e-5)) / np.pi) * 180   # 梯度方向
      if grad_x[i, j] < 0:
         grad_dir[i, j] = grad_dir[i, j]+180
      grad_dir[i, j] = (grad_dir[i, j] + 360) % 360
      grad_ori[i, j] = np.sqrt(grad_x[i, j] ** 2 + grad_y[i, j] ** 2) # 梯度幅值

  cellx_count = int(np.floor(cell_x))  # 向下取整
  celly_count = int(np.floor(cell_y))
  
  blockx_count = (figure_x - 2 * cell_x) / 2 + 1# 块的数量
  blocky_count = (figure_y - 2 * cell_y) / 2 + 1

  HOG = np.zeros((blocky_count, blockx_count, 2 * 2 * 8))#初始化HOG
  for y in xrange(blocky_count):   #全部覆盖一遍
    for x in xrange(blockx_count):
      block_h = np.zeros(32)
      block = grad[y * 2:y * 2 + 16, x * 2:x * 2 + 16]# 块
      for k in xrange(2):
        for m in xrange(2): 
          cell_h = np.zeros(8)
          cell = block[k * 8:(k + 1) * 8, m * 8:(m + 1) * 8]# cell
          for i in xrange(celly_count):
            for j in xrange(cellx_count):
              cell_h[int(cell[i, j, 0] / 45)] += cell[i, j, 1]
          block_h[(k *2 + m) * 8:(k * 2 + m + 1) * 8] = cell_h[:]
      HOG[y, x, :] = block_h / np.sqrt(block_h.sum() ** 2 + 1e-5)
  HOG = HOG.ravel()
  return HOG

In [12]:
def hog_feature_fun(images):
    Hog_images=np.zeros((len(images),))
    for i in range(len(images)):
        Hog_images[i]=hog_feature(images[i])
    return Hog_images

### color_histogram

In [13]:

def color_histogram_hsv(im, nbin=10, xmin=0, xmax=255, normalized=True):
  ndim = im.ndim
  bins = np.linspace(xmin, xmax, nbin+1)
  hsv = colors.rgb_to_hsv(im/xmax) * xmax
  imhist, bin_edges = np.histogram(hsv[:,:,0], bins=bins, density=normalized)
  imhist = imhist * np.diff(bin_edges) 
  return imhist

# color_histogram
num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats1 = extract_features(X_train, feature_fns, verbose=True)
X_val_feats1 = extract_features(X_val, feature_fns)
X_test_feats1 = extract_features(X_test, feature_fns)

# HOG
X_train_feats2 = hog_feature_fun(X_train)  
X_val_feats2 = hog_feature_fun(X_val)
X_test_feats2 = hog_feature_fun(X_test)

# Concatenating the HOG and color histogram feature vectors.
X_train_feats = np.concatenate((X_train_feats1,X_train_feats2),axis=1)  
X_val_feats =  np.concatenate((X_val_feats1,X_val_feats2), axis=1)
X_test_feats =  np.concatenate((X_test_feats1,X_test_feats2), axis=1)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat  
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
# In k-NN, the bias dimension is useless. But you may need it in other classifiers.
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

NameError: global name 'grad_dir' is not defined

## k-NN on features
Using the k-NN code developed earlier in the assignment. This should achieve better results than k-NN directly on top of raw pixels.

In [ ]:
from camalab.classifiers import KNearestNeighbor

classifier = KNearestNeighbor()
classifier.train(X_train_feats, y_train)

In [ ]:
# Use the validation set to tune the 'k'

k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]
k_to_accuracies = {} # your should store the results in this dict

################################################################################
# TODO:                                                                        #
# You must use the simple validation method. Because we have divided the       #
# dataset to validation set and train set.                                     #
################################################################################
pass
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

for k in sorted(k_to_accuracies):
    print 'k = %d, accuracy = %f' % (k, k_to_accuracies[k])

In [ ]:
# Evaluate your classifier

best_k = 1  # choose your best k

# Compute and display the accuracy
y_test_pred = classifier.predict(X_test_feats, k=best_k)
test_accuracy = np.mean(y_test == y_test_pred)
print test_accuracy